## Purpose of our Program: 
The purpose of our program is to connect readers so they can get recommendations, provide reviews, and find new books to read.

### Purpose of my feature:
The purpose of my feature, the wishlist, is for readers to be able to keep track of the books that they want to read in the future.

### Lists
Lists are used to handle rows of data, such as when restoring the Wishlist table from a list of dictionaries. Here, data is a list of dictionaries, each representing a row to be added to the database.
The restore method iterates over the list and adds each dictionary as a row in the database.

In [ ]:
data = [{"book_id": 1}, {"book_id": 2}, {"book_id": 3}]
Wishlist.restore(data)

### Dictionaries
Each dictionary contains column names as keys (id, book_id) and their respective values.
This is particularly useful when serializing database objects for use in APIs, such as returning JSON responses.

In [ ]:
def read(self):
    return {
        "id": self.id,
        "book_id": self.book_id,
    }

### CRUD
The function get_wishlist() retrieves all rows from the wishlist table as a list of Wishlist objects. The SQLAlchemy ORM is the third-party library used here.

In [ ]:
def get_wishlist():
    return Wishlist.query.all()

The returned list can be further processed, e.g., converted into a JSON response. This converts the list of Wishlist objects into a list of dictionaries. 


In [ ]:
wishlist_items = [item.read() for item in get_wishlist()]


The Wishlist model provides methods for handling individual columns (id, book_id):

Create: Using add_to_wishlist(book_id):


In [ ]:
item = Wishlist(book_id=book_id)
db.session.add(item)
db.session.commit()

Read: Using read():


In [ ]:
wishlist_dict = item.read()


Delete: Using delete_from_wishlist(book_id):


In [ ]:
item = Wishlist.query.filter_by(book_id=book_id).first()
db.session.delete(item)
db.session.commit()

Sequencing, Selection, and Iteration

The add_book_to_wishlist method demonstrates sequencing, selection, and iteration:






In [ ]:
# Route to add a book to the wishlist
@wishlist_api.route('/', methods=['POST'])
def add_book_to_wishlist():
    """Add a book to the wishlist."""
    if request.is_json:
        data = request.get_json()
        book_id = data.get('book_id')

        # Validate that book_id is provided
        if not book_id:
            return jsonify({"error": "Missing book_id"}), 400

        # Check if the book exists in the books database
        book = Book.query.get(book_id)
        if not book:
            return jsonify({"error": "Book not found"}), 404

        # Check if the book is already in the wishlist
        existing_entry = Wishlist.query.filter_by(book_id=book_id).first()
        if existing_entry:
            return jsonify({"message": "Book already in wishlist"}), 200

        # Add the book to the wishlist
        new_entry = Wishlist(book_id=book_id)
        db.session.add(new_entry)
        db.session.commit()

        return jsonify({"message": "Book added to wishlist"}), 201

    return jsonify({"error": "Request must be JSON"}), 415


Sequencing: Sequentially processes the request, validates the input, checks database records, and adds a new entry.

Selection: Uses if conditions to handle cases like missing book_id, non-existent books, or duplicate entries.

Iteration: Iterates through existing records using a loop to build the wishlist response.

Parameters and Return Types

Parameters:

The POST request body must be JSON and include `book_id`.

Example body:

{
  "book_id": 1
}

Return Type: JSON responses are created using `jsonify`, returning:

Success messages (`201` or `200` status).

Error messages (`400`, `404`, or `415` status).

## Making a Request


In [ ]:
// Adding a book to the wishlist
fetch('/api/wishlist/', {
    method: 'POST',
    headers: {
        'Content-Type': 'application/json'
    },
    body: JSON.stringify({ book_id: 1 })
})
.then(response => response.json())
.then(data => console.log(data));

## Testing API's on postman

Fetching all the books:
 {
        "author": "Charles Dickens",
        "id": 1,
        "title": "Great Expectations"
    },
    {
        "author": "S.E. Hinton",
        "id": 2,
        "title": "The Outsiders"
    },
...

Fetching the wishlist:
 {
        "author": "Jane Austen",
        "id": 4,
        "title": "Pride and Prejudice"
    },
    {
        "author": "Traci Chee",
        "id": 16,
        "title": "We are Not Free"
    },
...

Adding a book to wishlist:
{
    "message": "Book added to wishlist"
}
or
{
    "message": "Book already in wishlist"
}

Deleting book from wishlist:
{
    "message": "Book removed from wishlist"
}
or
{
    "error": "Book not found in wishlist"
}